<div align="center" dir="auto">
<p dir="auto"><a href="https://colab.research.google.com/drive/1nwdSHFonQBEYEhywLfQPkMMya0xTOjdc?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<a target="_blank" rel="noopener noreferrer nofollow" href="https://camo.githubusercontent.com/1a088dbe61584d1b213517c05e313c765ca127ce21118626b2d0db0b8f573235/68747470733a2f2f696d672e736869656c64732e696f2f6769746875622f6c6963656e73652f656e636f72642d7465616d2f656e636f72642d616374697665"><img src="https://camo.githubusercontent.com/1a088dbe61584d1b213517c05e313c765ca127ce21118626b2d0db0b8f573235/68747470733a2f2f696d672e736869656c64732e696f2f6769746875622f6c6963656e73652f656e636f72642d7465616d2f656e636f72642d616374697665" alt="Licence" data-canonical-src="https://img.shields.io/github/license/encord-team/encord-active" style="max-width: 100%;"></a>
<a href="https://www.piwheels.org/project/encord-active/" rel="nofollow"><img src="https://camo.githubusercontent.com/ad0f13bbb94beadb953a46e51cee0db17ade62e4deb8c306e36a7addfa69d18f/68747470733a2f2f696d672e736869656c64732e696f2f707970692f762f656e636f72642d616374697665" alt="PyPi project" data-canonical-src="https://img.shields.io/pypi/v/encord-active" style="max-width: 100%;"></a>
<a target="_blank" rel="noopener noreferrer nofollow" href="https://camo.githubusercontent.com/890ceffab6602e1a6ecac875e179b3984708f98b0a91293e6c632752d11bad7e/68747470733a2f2f696d672e736869656c64732e696f2f707970692f707976657273696f6e732f656e636f72642d616374697665"><img src="https://camo.githubusercontent.com/890ceffab6602e1a6ecac875e179b3984708f98b0a91293e6c632752d11bad7e/68747470733a2f2f696d672e736869656c64732e696f2f707970692f707976657273696f6e732f656e636f72642d616374697665" alt="PyPi version" data-canonical-src="https://img.shields.io/pypi/pyversions/encord-active" style="max-width: 100%;"></a>

<a href="https://docs.encord.com/active/docs" rel="nofollow"><img src="https://camo.githubusercontent.com/457447b616a4e5aecccd5351fa78d41ce828901e395b935ad2a94d94db75e5d0/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f646f63732d6f6e6c696e652d626c7565" alt="docs" data-canonical-src="https://img.shields.io/badge/docs-online-blue" style="max-width: 100%;"></a>
<a href="https://discord.gg/TU6yT7Uvx3" rel="nofollow"><img src="https://camo.githubusercontent.com/927173d1e92c30f5e17607e8d270d681c6a665591ba93475b070f4e7f5d2e0e9/68747470733a2f2f736869656c64732e696f2f62616467652f446973636f72642d636861742d626c75653f6c6f676f3d646973636f7264266c6f676f436f6c6f723d7768697465" alt="&quot;Join us on Discord&quot;" data-canonical-src="https://shields.io/badge/Discord-chat-blue?logo=discord&amp;logoColor=white" style="max-width: 100%;"></a>
<a href="https://github.com/encord-team/encord-notebooks" rel="nofollow"><img src="https://badges.aleen42.com/src/github.svg" alt="&quot;Encord Notebooks;" data-canonical-src="https://badges.aleen42.com/src/github.svg" style="max-width: 100%;"></a>


</p>
<p dir="auto">
<a href="https://twitter.com/encord_team" rel="nofollow"><img src="https://camo.githubusercontent.com/c7ea37e4d1f0c2e900069152f5990d3bcef4a69c97dede26f08d588958410bd8/68747470733a2f2f696d672e736869656c64732e696f2f747769747465722f666f6c6c6f772f656e636f72645f7465616d3f6c6162656c3d253430656e636f72645f7465616d267374796c653d736f6369616c" alt="Twitter Follow" data-canonical-src="https://img.shields.io/twitter/follow/encord_team?label=%40encord_team&amp;style=social" style="max-width: 100%;"></a></p>
</div>

<div align="center">
  <p>
    <a align="center" href="" target="_blank">
      <img
        width="7232"
        src="https://storage.googleapis.com/encord-notebooks/encord_active_notebook_banner.png">
    </a>
  </p>
</div>

# 🟣 Encord Notebooks | 📦 Generate Encord Active Model Object Detection Boxes Using Encord Annotate Micro-Models

## 🏁 Overview

👋 Hi there!

Do you know how to build micro-models? Great! In this short notebook walkthrough, learn how to generate bounding boxes with 🟣 Encord Active using Micro-Models in Encord Annotate.

> Note: [Ontologies](https://docs.encord.com/ontologies/use/) of the Encord Active and Encord Annotate should be the same

<br>

💡If you want to read more about 🟣 Encord Active checkout our [GitHub](https://github.com/encord-team/encord-active) and [documentation](https://encord-active-docs.web.app/).


## 📐 Set parameters

In [ ]:
ENCORD_ACTIVE_PROJECT_PATH='</path/to/local/encord_active/project>' # Enter the path to your encord project

# Store your environment variables
ENCORD_SSH_KEY_PATH='ENCORD_SSH_KEY_PATH'
ENCORD_PROJECT_HASH='ENCORD_PROJECT_HASH'
ENCORD_MODEL_ITERATION_HASH='ENCORD_MODEL_ITERATION_HASH'

BATCH_SIZE=24
CONFIDENCE_THRESHOLD=0.6
IOU_THRESHOLD=0.3

## 📥 Import libraries

In this section, you will import the key libraries that will be used for generating the bounding boxes. These libraries play a crucial role in executing the code examples and demonstrating the concepts covered in the walkthrough.

In [ ]:
import os
import json
import pickle
from pathlib import Path

from encord import EncordUserClient, Project
from tqdm import tqdm

from encord_active.lib.common.iterator import DatasetIterator
from encord_active.lib.db.predictions import (
    BoundingBox,
    Format,
    ObjectDetection,
    Prediction,
)
from encord_active.lib.project.project_file_structure import ProjectFileStructure

Now, you will generate bounding boxes for an Encord Active project using micro-models. You will establish a connection to the Encord Active platform using the SSH private key you stored as an env variable. Then, retrieve the project information using the project hash. and load the ontology information from the project.

Use an iterator to iterate over the dataset in your project, and use the micro-model in Encord Annotate to generate segmentation masks. The masks will be processed, filtered based on shape and ontology availability, and stored as `Prediction` objects. Finally, the box co-ordinates/points (predictions) will be saved to a pickle file and imported into the Encord Active platform so you can evaluate the quality.

In [ ]:
# Establish a connection to the Encord Active platform using SSH private key
user_client = EncordUserClient.create_with_ssh_private_key(Path(ENCORD_SSH_KEY_PATH).expanduser().read_text())

# Retrieve the Encord Active project
project: Project = user_client.get_project(ENCORD_PROJECT_HASH)

# Set up the project file structure for the Encord Active project
ea_project_fs: ProjectFileStructure = ProjectFileStructure(ENCORD_ACTIVE_PROJECT_PATH)

# Create a dataset iterator for the project directory
iterator = DatasetIterator(ea_project_fs.project_dir)

# Load the ontology information from the project
ontology = json.loads(ea_project_fs.ontology.read_text(encoding="utf-8"))
ontology_featureHashes = [obj["featureNodeHash"] for obj in ontology.get("objects")]

# Initialize lists to store predictions, file paths, and data units
predictions_to_store = []
file_paths = []
data_units = []

pbar = tqdm(total=iterator.length, desc="Running inference", leave=True)

# Iterate over the dataset
for counter, (data_unit, _) in enumerate(iterator.iterate()):

    # Append the file path corresponding to the data unit
    file_paths.append(
        next(
            ea_project_fs.label_row_structure(iterator.label_hash).iter_data_unit(data_unit["data_hash"])
        ).path.as_posix()
    )
    data_units.append(data_unit)

    # Perform inference in batches or at the end of the iteration
    if (counter + 1) % BATCH_SIZE == 0 or counter + 1 == iterator.length:

        # Perform model inference using the Encord Active project and specified parameters
        inference_results = project.model_inference(
            ENCORD_MODEL_ITERATION_HASH,
            file_paths=file_paths,
            conf_thresh=CONFIDENCE_THRESHOLD,
            iou_thresh=IOU_THRESHOLD,
        )

        # Process the inference results for each data unit
        for inference_result, du in zip(inference_results, data_units):

            # Iterate over the predicted objects
            for obj in inference_result["predictions"]["0"]["objects"]:
                
                # Skip predictions that are not in the 'bounding_box' shape
                if obj["shape"] != "bounding_box":
                    print(f"prediction is not in `bbox` type (it is in {obj['shape']} type), skipping...")
                    continue
                
                # Skip predictions that are not present in the ontology
                if obj["featureHash"] not in ontology_featureHashes:
                    print(
                        f"'{obj['name']}' with featureHash '{obj['featureHash']}' is not available in the ontology of"
                        f" the Encord Active project."
                    )
                    continue

                # Create a Prediction object with bounding box information        
                prediction = Prediction(
                    data_hash=du["data_hash"],
                    confidence=obj["confidence"],
                    object=ObjectDetection(
                        format=Format.BOUNDING_BOX,
                        data=BoundingBox(
                            x=obj["bounding_box"]["x"],
                            y=obj["bounding_box"]["y"],
                            w=obj["bounding_box"]["w"],
                            h=obj["bounding_box"]["h"],
                        ),
                        feature_hash=obj["featureHash"],
                    ),
                )
                predictions_to_store.append(prediction)
        
        # Clear the file paths and data units lists for the next batch
        file_paths = []
        data_units = []

    pbar.update(1)

# Define the filename for the prediction pickle file
prediction_file = f"predictions_{ENCORD_MODEL_ITERATION_HASH[:8]}.pkl"

# Save the predictions to a pickle file
with open((ea_project_fs.project_dir / prediction_file), "wb") as f:
    pickle.dump(predictions_to_store, f)

In [ ]:
# Change the current working directory to the project directory
os.chdir(ea_project_fs.project_dir.as_posix())

# Import the predictions file into Encord Active
! encord-active import predictions {prediction_file}

print("\n✅︎ Predictions are imported successfully!")

# ✅ Wrap up


📓This Colab notebook showed you how to generate detection boxes with 🟣 Encord Active using Micro-Models in Encord Annotate.

---

🟣 Encord Active is an open-source framework for computer vision model testing, evaluation, and validation. Check out the project on [GitHub](https://github.com/encord-team/encord-active), leave a star 🌟 if you like it, and leave an issue if you find something is missing.

---

👉 Check out our 📖[blog](https://encord.com/blog/webinar-semantic-visual-search-chatgpt-clip/) and 📺[YouTube](https://www.youtube.com/@encord) channel to stay up-to-date with the latest in computer vision, foundation models, active learning, and data-centric AI.



### ✨ Want more walthroughs like this? Check out the 🟣 [Encord Notebooks repository](https://github.com/encord-team/encord-notebooks/).